In [1]:
import os
import shutil
import pandas as pd

In [2]:
# dataDicts = []
# for root, dirs, files in os.walk('oasis3downloads'):
#     for file in files:
#         if 'nii.gz' in file:
#             filepath = os.path.join(root,file)
#             if '_MR_' in filepath:
#                 type = str(filepath).split('_')[-1]
#                 type = type.split('.nii.gz')[0]
#                 baseFolder = str(filepath).split(os.sep)[1]
#                 newPath = os.path.join('NIFTI',baseFolder,file)
#                 temp = pd.Series()
#                 temp = {'origpath':filepath, 'type':type, 'newpath':newPath}
#                 dataDicts.append(temp)
#             break

In [3]:
# df = pd.read_csv('MoveFileTest.csv', index_col=0)
# df.head(10)

In [5]:
# print(df['type'].unique())

['NotMri' 'bold' 'T2w' 'T1w' 'T2star' 'minIP' 'dwi' 'swi' 'FLAIR' 'GRE'
 'asl' 'fieldmap' 'angio' 'pasl' nan]


In [52]:
labeldict = {"T1":0,"T1_c":1,"T2":2,"T2-FLAIR":3,"FS":4,"PD":5,
             "SWI":6, "GRE":7, "T2*":8, "DWI":9, "ADC":10, "BOLD":11,
             "FlowSensitve":12,"PWI":13,"Other":14}
def assignLabel(dataframe):
    description = dataframe['description']
    try:
        weight = dataframe['weighting']
    except:
        weight = None
    otherCategories = ['fieldmap','scout','hippo','calibration','phase','localizer','asl','dti','fspgr','cbf', 'nan', 'pasl','NotMri','pet','CT','e2']
    if description in otherCategories:
        return 14
    if description =='T1w':
        return 0
    if description =='T1c':
        return 1
    if weight:
        if description =='T2w':
            if weight=='T2':
                return 2
            if weight=='PD':
                return 5
    if description == 'T2w':
        return 2
    if 'FLAIR' in description:
        return 3
    if description =='FS':
        return 4
    if description =='pd':
        return 5
    if description =='swi' or description=='minIP':
        return 6
    if description.lower() =='gre':
        return 7
    if description =='T2star':
        return 8
    if description =='dwi':
        return 9
    if description =='adc':
        return 10
    if description =='bold':
        return 11
    if description =='angio': #flowsensitive
        return 12
    if description =='pwi':
        return 13
    return -1

In [60]:
Labels = pd.read_csv('Labels.txt', names=['ID','label','extra'],sep='\t')

In [61]:
Labels

,ID,label,extra
0,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS300...,0,0
1,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS304...,0,0
2,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS306...,0,0
3,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS306...,0,0
4,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS302...,0,0
...,...,...,...
53270,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS304...,0,0
53271,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS312...,0,0
53272,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS304...,0,0
53273,data/OASIS/OASIS3/test/NIFTI_SLICES/sub-OAS300...,0,0


In [62]:
def findType(ID):
    type = ID.split('__')[0]
    type = type.split('_')[-1]
    return type

In [63]:
Labels['description'] = Labels['ID'].apply(findType)

In [64]:
print(Labels.columns)

Index(['ID', 'label', 'extra', 'description'], dtype='object')


In [65]:
Labels['label'] = Labels.apply(assignLabel,axis=1)
Labels['label'].value_counts()

label
 14    14325
 11     7850
 0      6375
 2      6375
 9      5200
 6      3800
 8      3625
 3      2125
 7      2000
 12     1575
-1        25
Name: count, dtype: int64

In [66]:
Labels[['ID','label','extra']].to_csv('Test_labels_annotated.txt',index=False,sep='\t')